In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


In [2]:
my_images = []
labels = [0]*5 + [1]*5

for i in range(10):
    file = f"./my_images/img{i+1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    my_images.append(image)

X = np.array(my_images, dtype='float32') / 255.0
y = np.array(labels)


In [3]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [4]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    fill_mode='nearest'
)
datagen.fit(X_train)


In [5]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(96,96,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])


C:\Users\yhye1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [7]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)


In [8]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=2),
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=[early_stop]
)


Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - accuracy: 0.7500 - loss: 0.6915 - val_accuracy: 0.0000e+00 - val_loss: 0.7037
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5000 - loss: 0.6898 - val_accuracy: 0.5000 - val_loss: 0.7082
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6250 - loss: 0.6922 - val_accuracy: 0.5000 - val_loss: 0.7090
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.6936 - val_accuracy: 0.5000 - val_loss: 0.7156
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6250 - loss: 0.6887 - val_accuracy: 0.5000 - val_loss: 0.7179
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5000 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.7204
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5000 - loss: 0.6943 - val_accuracy: 0.5000 - val_loss: 0.7374
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5000 - loss: 0.6929 - val_accuracy: 0.5000 - val_

In [9]:
model.save("MY_DETECTOR_SMALLCNN.keras")


In [10]:
test_images = []
for i in range(10):
    file = f"./test_images/img{i+1:02d}.jpg"
    image = cv.imread(file)
    image = cv.resize(image, (96, 96))
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    test_images.append(image)

test_images_arr = np.array(test_images, dtype='float32') / 255.0


In [11]:
cnn_model = keras.models.load_model("MY_DETECTOR_SMALLCNN.keras")
predictions = cnn_model.predict(test_images_arr)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


In [12]:
for i, p in enumerate(predictions):
    if p > 0.5:
        print(f"img{i+1:02d}.jpg → ✅ 내 얼굴 ({p[0]:.3f})")
    else:
        print(f"img{i+1:02d}.jpg → ❌ 타인 ({p[0]:.3f})")


img01.jpg → ❌ 타인 (0.483)
img02.jpg → ✅ 내 얼굴 (0.531)
img03.jpg → ❌ 타인 (0.478)
img04.jpg → ✅ 내 얼굴 (0.506)
img05.jpg → ✅ 내 얼굴 (0.502)
img06.jpg → ✅ 내 얼굴 (0.512)
img07.jpg → ✅ 내 얼굴 (0.518)
img08.jpg → ✅ 내 얼굴 (0.524)
img09.jpg → ✅ 내 얼굴 (0.509)
img10.jpg → ❌ 타인 (0.498)
